<a href="https://colab.research.google.com/github/sevaroy/play-notebbok/blob/main/Prophet_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tqdm
!pip install pandas
!pip install prophet
!pip install yfinance
!pip install plotly
!pip install datetime
!pip install numpy

     |████████████████████████████████| 65 kB 3.1 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6640084 sha256=561e4f61975881c460c82af9b6cd8879f2f5f95ea25b8ff31300a65cb9e0a1bf
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 6.4 MB 16.3 M

In [4]:
# Libraries
from tqdm import tqdm
import pandas as pd
from prophet import Prophet
import yfinance as yf
from datetime import datetime, timedelta
import plotly.express as px
import numpy as np

In [5]:
# Time periods
now = datetime.now()

# How far back to retrieve
ago = now - timedelta(days=730)

# Designating the Ticker
crypto = yf.Ticker("BTC-USD")

# Getting price history
df = crypto.history(start=ago.strftime("%Y-%m-%d"), end=now.strftime("%Y-%m-%d"), interval="1d")

# Handling missing data from yahoo finance
df = df.reindex(
    [df.index.min()+pd.offsets.Day(i) for i in range(df.shape[0])],
    fill_value=None
).fillna(method='ffill')

In [6]:
# Getting the N Day Moving Average and rounding the values
df['MA'] = df[['Open']].rolling(window=5).mean().apply(lambda x: round(x, 2))

# Dropping the NaNs
df.dropna(inplace=True)

In [7]:
# Formatted
df = df.reset_index().rename(columns={"Date": "ds", "MA": "y"})
df

,ds,Open,High,Low,Close,Volume,Dividends,Stock Splits,y
0,2020-03-30,5925.538574,6517.195801,5903.234375,6429.841797,37101651525,0,0,6406.60
1,2020-03-31,6430.606445,6504.515137,6374.162109,6438.644531,32786468812,0,0,6357.68
2,2020-04-01,6437.319336,6612.573730,6202.373535,6606.776367,40346426266,0,0,6301.27
3,2020-04-02,6606.776367,7088.247559,6595.918457,6793.624512,47660646124,0,0,6329.17
4,2020-04-03,6797.396484,7003.220703,6673.335938,6733.387207,38976504903,0,0,6439.53
...,...,...,...,...,...,...,...,...,...
722,2022-03-22,41074.105469,43124.707031,40948.281250,42358.808594,32004652376,0,0,41450.23
723,2022-03-23,42364.378906,42893.507812,41877.507812,42892.957031,25242943069,0,0,41734.13
724,2022-03-24,42886.652344,44131.855469,42726.164062,43960.933594,31042992291,0,0,41952.54
725,2022-03-25,43964.546875,44999.492188,43706.285156,44348.730469,30574413034,0,0,42307.16


In [9]:
# Setting up Prophet
m = Prophet(
    daily_seasonality=True, 
    yearly_seasonality=True, 
    weekly_seasonality=True
)

# Fitting to the prices
m.fit(df[['ds', 'y']])

In [10]:
# Future DF of 3 months
future = m.make_future_dataframe(periods=90)

In [11]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2020-03-30,4655.355603,1754.611271,7311.985728
1,2020-03-31,5047.770432,2327.647110,7872.869730
2,2020-04-01,5407.644238,2497.785051,8022.426111
3,2020-04-02,5750.112702,2870.235025,8484.469450
4,2020-04-03,6013.944695,3371.087443,8852.489324
...,...,...,...,...
812,2022-06-20,-20103.679715,-27034.677061,-11858.736980
813,2022-06-21,-20671.676762,-28235.728146,-12441.302868
814,2022-06-22,-21236.991779,-28636.978025,-13007.126015
815,2022-06-23,-21786.682741,-29627.004383,-13502.865749


In [12]:
# Visual DF
vis_df = df[['ds','Open']].append(
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
).rename(columns={'yhat': 'Prediction'})

# Visualizing results
fig = px.line(
    vis_df,
    x='ds',
    y=['Open', 'Prediction', 'yhat_lower', 'yhat_upper'],
    title='Crypto Forecast',
    labels={'value':'Price',
            'ds': 'Date'}
)

# Adding a slider
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

In [13]:
def getData(ticker, window, ma_period):
    """
    Grabs price data from a given ticker. Retrieves prices based on the given time window; from now
    to N days ago.  Sets the moving average period for prediction. Returns a preprocessed DF
    formatted for FB Prophet.
    """
    # Time periods
    now = datetime.now()

    # How far back to retrieve tweets
    ago = now - timedelta(days=window)

    # Designating the Ticker
    crypto = yf.Ticker(ticker)

    # Getting price history
    df = crypto.history(start=ago.strftime("%Y-%m-%d"), end=now.strftime("%Y-%m-%d"), interval="1d")
    
    # Handling missing data from yahoo finance
    df = df.reindex(
        [df.index.min()+pd.offsets.Day(i) for i in range(df.shape[0])],
        fill_value=None
    ).fillna(method='ffill')
    
    # Getting the N Day Moving Average and rounding the values
    df['MA'] = df[['Open']].rolling(window=ma_period).mean().apply(lambda x: round(x, 2))

    # Dropping the NaNs
    df.dropna(inplace=True)

    # Formatted for FB Prophet
    df = df.reset_index().rename(columns={"Date": "ds", "MA": "y"})
    
    return df

In [14]:
def fbpTrainPredict(df, forecast_period):
    """
    Uses FB Prophet and fits to a appropriately formatted DF. Makes a prediction N days into 
    the future based on given forecast period. Returns predicted values as a DF.
    """
    # Setting up prophet
    m = Prophet(
        daily_seasonality=True, 
        yearly_seasonality=True, 
        weekly_seasonality=True
    )
    
    # Fitting to the prices
    m.fit(df[['ds', 'y']])
    
    # Future DF
    future = m.make_future_dataframe(periods=forecast_period)
        
    # Predicting values
    forecast = m.predict(future)

    # Returning a set of predicted values
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [16]:
def visFBP(df, forecast):
    """
    Given two dataframes: before training df and a forecast df, returns
    a visual chart of the predicted values and actual values.
    """
    # Visual DF
    vis_df = df[['ds','Open']].append(forecast).rename(
        columns={'yhat': 'Prediction',
                 'yhat_upper': "Predicted High",
                 'yhat_lower': "Predicted Low"}
    )

    # Visualizing results
    fig = px.line(
        vis_df,
        x='ds',
        y=['Open', 'Prediction', 'Predicted High', 'Predicted Low'],
        title='Crypto Forecast',
        labels={'value':'Price',
                'ds': 'Date'}
    )

    # Adding a slider
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    return fig.show()

In [17]:
# Getting and Formatting Data
df = getData("BTC-USD", window=365, ma_period=3)

# Training and Predicting Data
forecast = fbpTrainPredict(df, forecast_period=21)

# Visualizing Data
visFBP(df, forecast)

In [18]:
def runningFBP(ticker, window=730, ma_period=5, days_to_train=365, forecast_period=10):
    """
    Runs the facebook prophet model over the provided ticker.  Trains with last N days given 
    by days_to_train.  Forecast N days into the future based on given forecast_period.  Moving average 
    is applied to the dataset based on given ma_period. Returns the root mean squared error and a DF 
    of the actual values and the predicted values for the same day.
    """

    # Getting and Formatting Data
    df = getData(ticker, window=window, ma_period=ma_period)
    
    # DF for the predicted values
    pred_df = pd.DataFrame()

    # Running the model on each day
    for i in tqdm(range(days_to_train, window-forecast_period, forecast_period)):

        # Training and Predicting the last day on the forecast
        forecast = fbpTrainPredict(df[i-days_to_train:i], 
                                   forecast_period=forecast_period).tail(forecast_period)[['ds',
                                                                                           'yhat',
                                                                                           'yhat_lower',
                                                                                           'yhat_upper']]

        # Adding the last day predicted
        pred_df = pred_df.append(forecast, ignore_index=True)
        
    # Combining the predicted df and original df
    comb_df = df[['ds', 'Open']].merge(pred_df, 
                                       on='ds', 
                                       how='outer').sort_values(by='ds')
    
    # Setting the index to the dates
    comb_df.set_index('ds', inplace=True)

    return comb_df

In [19]:
def get_prophet_positions(df, short=True):
    """
    For these positions, buy when actual value is above the upper bound and short 
    when actual value is below lower bound. Otherwise do nothing.
    """
    if df['Open'] >= df['yhat_upper']:
        return 1
    elif df['Open'] <= df['yhat_lower'] and short:
        return -1
    else:
        return 0

In [20]:
def fbpBacktest(df, short=True):
    """
    Performs the final backtest using log returns and the positions function.
    Returns the performance.
    """
    # Getting positions
    df['positions'] = df.apply(lambda x: get_prophet_positions(x, short=short), axis=1)

    # Compensating for lookahead bias
    df['positions'] = df['positions'].shift(1)
    
    # Getting log returns
    df['log_returns'] = df['Open'].apply(np.log).diff()

    # Dropping any Nans
    df.dropna(inplace=True)
    
    # Performing the backtest
    returns = df['positions'] * df['log_returns']

    # Inversing the log returns and getting daily portfolio balance
    performance = returns.cumsum().apply(np.exp)
    
    return performance

In [21]:
# Running the model and getting forecast
bt_df = runningFBP("ETH-USD", 
                   window=730, 
                   ma_period=5, 
                   days_to_train=370, 
                   forecast_period=10)

# Performing the backtest
performance = fbpBacktest(bt_df, short=False)

# Visualizing results
px.line(performance,
        x=performance.index,
        y=performance,
        title='Portfolio Performance',
        labels={"y": "Portfolio Balance",
                "ds": "Date"})

100%|██████████| 35/35 [01:36<00:00,  2.75s/it]


In [22]:
bt_df


,Open,yhat,yhat_lower,yhat_upper,positions,log_returns
ds,,,,,,
2021-04-04,2027.671265,1780.582180,1711.954912,1847.838585,0.0,-0.055270
2021-04-05,2093.260742,1775.726698,1714.321285,1839.948011,1.0,0.031835
2021-04-06,2109.493164,1771.221967,1700.175476,1841.746379,1.0,0.007725
2021-04-07,2117.728516,1767.417152,1700.116001,1835.534215,1.0,0.003896
2021-04-08,1969.133179,1764.302325,1692.597172,1830.251778,1.0,-0.072751
...,...,...,...,...,...,...
2022-03-15,2590.668945,2620.799438,2400.186087,2846.106065,0.0,0.028258
2022-03-16,2620.028564,2632.422746,2420.051680,2872.999126,0.0,0.011269
2022-03-17,2771.964111,2656.209700,2435.927744,2888.289757,0.0,0.056371
